<h1>Train Neural Network To Estimate Average Strenghts and Variations on the Flop </h1>
There was an error in the evaluator for average hand strengths in the PokerAI-Agent. We must therefore retrain the mode. The error came about in generating the training data. 
Beyond correcting the error, we want to change the outputs of the model. We want to leave the outputs at 3, but instead of outputting average win and average loss, which are actually identical, we will generate two estimates for deviations - a std for win/loss and a std for split. Overall there will be five estimators: Pwin, Ploose(or Psplit?), pwinVillain, stdWinVillain, and stdsplit.

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from keras import backend as K
import random
dataFolder = 'simulatedHands/'
modelsFolder = 'trainedModels/strengthEvaluator/'

<h2>Simulated boards</h2>
We can still use 80 thousand simulated hands form the simHandsTurn-Data Set. The error was not to be found in the estimates for Hero-wins and losses. The data frame consists of hero hands and boards, as well as probabilities of winning, loosing, or splitting the hand against random opponent hands.

In [3]:
simHandsFlop = pd.read_csv(dataFolder +  'simHandsFlop.csv', index_col=0)
simHandsFlop.drop(['tval', 'tsuit','rval', 'rsuit'], axis=1, inplace=True)
simHandsFlop.shape

(80000, 13)

In [3]:
simHandsFlop.head()

,val1,suit1,val2,suit2,f1val,f1suit,f2val,f2suit,f3val,f3suit,nwin,nloose,nsplit
0,4,2,8,2,2,3,9,3,10,1,0.2120,0.7422,0.0458
1,5,2,10,3,8,3,11,2,13,0,0.3080,0.6248,0.0672
2,12,1,13,3,10,0,10,2,12,2,0.7650,0.2038,0.0312
3,7,0,10,3,2,1,11,0,13,1,0.3252,0.6280,0.0468
4,10,2,14,3,2,2,7,1,13,2,0.5306,0.4516,0.0178


Let's check how accurate those percentages are, running a few of those lines against all possible villains.

In [5]:
from itertools import combinations
from PokerHandStrengths import compare
#create a deck
deck=[]
for val in range(2,15):
    for suit in range(4):
        deck.append((val,suit))


In [6]:
#function that takes in a row from the dataFrame and creates the list for hands usable in the compare function. The compare function, that chesk which hand won, needs lists of tuples as input.
def transformHand(row):
    rowInt = row.astype(int)
    hero = [(rowInt[0], rowInt[1]), (rowInt[2],rowInt[3])]
    board = [(rowInt[4], rowInt[5]), (rowInt[6],rowInt[7]),(rowInt[8], rowInt[9])]
    return hero + board

80 tousand boards might not be enough to train on. Let's create more boards.

In [158]:
#simulate boards
boards=[]
for i in range(160000):
    board = np.array(random.sample(deck,5)).flatten()
    boards.append(list(board))
boards=np.array(boards)
boards

array([[13,  3,  2, ...,  1,  8,  0],
       [12,  2,  8, ...,  0,  7,  2],
       [ 2,  0,  6, ...,  3,  4,  3],
       ...,
       [14,  0,  3, ...,  0, 12,  0],
       [ 3,  2,  8, ...,  2,  5,  3],
       [ 6,  3,  8, ...,  3,  4,  0]])

and simulate nwin, nloose, and nsplit

In [7]:
#This function takes in the deck, the board and simulates pwin, plosse and psplit. The firs four entries in one array are hero's cards. The following six represent the flop.
#The function randomly draws villain cards and turns and rivers. It compares hero and villains and returns avarage wins, losses and splits for hero.
def simulate(deckOfCards, board, nsim = 2000):
    import random
    nwin = 0
    nloose = 0
    nsplit = 0
    boardTransformed = transformHand(board)
    deckRemaining = set(deckOfCards)- set(boardTransformed)
    for i in range(nsim):
        sample = random.sample(deckRemaining, 4)
        handHero = boardTransformed + sample[2:]
        handVillain = sample[:2] +  boardTransformed[2:] + sample[2:]
        result = compare({0:handHero, 1:handVillain})
        if result[0]<result[1]:
            nwin+=1
        elif result[1]<result[0]:
            nloose += 1
        else:
            nsplit+=1
    return nwin/nsim, nloose/nsim, nsplit/nsim
        

In [8]:
#To increase speed we want to simulate runouts on 8 threads simulatanously. The map.sync-function from ipyparallel needs one fucniton with number of threads as parameter.
# The function will return simulated results for datasets given to all threads.
def simulateMulti(nThreads):
    import numpy as np #must import numpy in the function, because it's not possible to give the numpy module to the threads manually.
    results = []
    for i in range(len(dataset)):
        results.append(simulate(deck, dataset[i], nsim=1000))
    return results

In [224]:
#create a cluster of threads and start
nThreads = 8
import ipyparallel as ipp
cluster = ipp.Cluster(n=nThreads)
await cluster.start_cluster()

Starting 8 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


<Cluster(cluster_id='1641816621-vw7m', profile='default', controller=<running>, engine_sets=['1641816623'])>

In [225]:
rc = cluster.connect_client_sync()
rc.wait_for_engines(n=8)

100%|█████████████████████████████████████████| 8/8 [00:07<00:00,  1.01engine/s]


In [226]:
#Let's give the threads the necessary objects.
rc[:]['simulate']=simulate #simulate-function
rc[:]['transformHand']=transformHand #transform Hand function
rc[:]['deck']=deck
rc[:]['compare']=compare
for i in range(8):
    rc[i]['dataset']=boards[len(boards)//8*i:len(boards)//8*i+len(boards)//8] #eacth thread receives one eigth of the dataset.

In [227]:
results = rc[:].map_sync(simulateMulti,range(8))


Let's combine the boards with the respective results.

In [259]:
len(results[0])
    

20000

In [254]:
dfBoards = pd.DataFrame(columns=simHandsFlop.columns[:-3], data = boards)
dfBoards.head()


,val1,suit1,val2,suit2,f1val,f1suit,f2val,f2suit,f3val,f3suit
0,13,3,2,1,3,3,5,1,8,0
1,12,2,8,3,6,3,12,0,7,2
2,2,0,6,1,5,0,12,3,4,3
3,5,2,3,3,2,3,14,2,4,2
4,9,2,12,0,9,1,3,0,6,2


In [281]:
dfResults = pd.DataFrame(columns=simHandsFlop.columns[-3:], data = results[0])
for i in range(1,len(results)):
    df = pd.DataFrame(columns=simHandsFlop.columns[-3:], data = results[i])
    dfResults = dfResults.append(df, ignore_index=True)
dfResults.shape               

(160000, 3)

concat the two data frames.

In [282]:
simHandsFlopNew = pd.concat([dfBoards,dfResults], axis=1, ignore_index=True)
simHandsFlopNew.columns = simHandsFlop.columns
simHandsFlopNew.shape

(160000, 13)

In [283]:
simHandsFlopNew.head()

,val1,suit1,val2,suit2,f1val,f1suit,f2val,f2suit,f3val,f3suit,nwin,nloose,nsplit
0,13,3,2,1,3,3,5,1,8,0,0.339,0.632,0.029
1,12,2,8,3,6,3,12,0,7,2,0.809,0.178,0.013
2,2,0,6,1,5,0,12,3,4,3,0.279,0.692,0.029
3,5,2,3,3,2,3,14,2,4,2,0.914,0.046,0.040
4,9,2,12,0,9,1,3,0,6,2,0.754,0.237,0.009


and finally lets add the initial 80k hands to the dataframe.

In [286]:
simHandsFlopLarge = simHandsFlopNew.append(simHandsFlop, ignore_index=True)
simHandsFlopLarge

,val1,suit1,val2,suit2,f1val,f1suit,f2val,f2suit,f3val,f3suit,nwin,nloose,nsplit
0,13,3,2,1,3,3,5,1,8,0,0.3390,0.6320,0.0290
1,12,2,8,3,6,3,12,0,7,2,0.8090,0.1780,0.0130
2,2,0,6,1,5,0,12,3,4,3,0.2790,0.6920,0.0290
3,5,2,3,3,2,3,14,2,4,2,0.9140,0.0460,0.0400
4,9,2,12,0,9,1,3,0,6,2,0.7540,0.2370,0.0090
...,...,...,...,...,...,...,...,...,...,...,...,...,...
239995,11,3,12,1,5,1,5,2,9,3,0.4462,0.5020,0.0518
239996,6,2,7,2,2,3,10,3,12,3,0.1470,0.7866,0.0664
239997,12,0,14,1,6,2,8,2,10,0,0.4640,0.5140,0.0220
239998,8,0,13,1,4,2,9,3,10,3,0.3910,0.5816,0.0274


In [287]:
simHandsFlopLarge.to_csv(dataFolder+'simHandsFlopLarge.csv', index= None)

In [7]:
simHandsFlopLarge = pd.read_csv(dataFolder+'simHandsFlopLarge.csv')

We are going to train a model to predict pwin, ploose, and psplit. It would take too long to simulate Average villain strengths and std for 95% of the boards. We need a very precise model, because we are going to use the results of this model to train another model, which estimates all five values.

In [9]:
input_dims = 10

fc1_dims = 128
fc2_dims = 64
fc3_dims = 32
fc4_dims = 16
fc5_dims = 8
fc6_dims = 4

lr = 0.0001
modelFlopNew = keras.Sequential()
modelFlopNew.add(keras.layers.Dense(fc1_dims, input_dim=input_dims, activation = 'relu'))
modelFlopNew.add(keras.layers.Dense(fc2_dims, activation = 'relu'))
modelFlopNew.add(keras.layers.Dense(fc3_dims, activation = 'relu'))
modelFlopNew.add(keras.layers.Dense(fc4_dims, activation = 'relu'))
modelFlopNew.add(keras.layers.Dense(fc5_dims, activation = 'relu'))
modelFlopNew.add(keras.layers.Dense(fc6_dims, activation = 'relu'))
modelFlopNew.add(keras.layers.Dense(3,activation='softmax')) #pwin, ploose and pplit must add up to 1.
modelFlopNew.compile(optimizer=Adam(learning_rate=lr), loss='mean_squared_error')

2022-01-10 16:10:32.557892: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-10 16:10:32.557984: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-10 16:10:32.558025: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cromi-Lenovo-V15-ADA): /proc/driver/nvidia/version does not exist
2022-01-10 16:10:32.560763: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(simHandsFlopLarge, test_size=0.05)

In [12]:
X_train = np.array(train.iloc[:,0:-3])
X_test = np.array(test.iloc[:,0:-3])
y_train = np.array(train.iloc[:,-3:])
y_test = np.array(test.iloc[:,-3:])

In [ ]:
lr = 0.00000001
K.set_value(modelFlopNew.optimizer.learning_rate, lr)
modelFlopNew.fit(X_train, y_train, epochs = 20, batch_size=8)

In [56]:
#Hopefully we did not overfit.
modelFlopNew.evaluate(X_test,y_test)

375/375 [==============================] - 1s 2ms/step - loss: 6.3473e-04


0.000634727708529681

In [57]:
modelFlopNew.save(modelsFolder+'strengthFlopCorrected.h5')

Let's generate a model.

<h2>Simulating average hand strengths for training data </h2>
Now that we have a propper network to evaluate individual hand strengths, let's generate training data for average hand strengths. Instead of providing one single hand for hero, we will use all possible combinations, estimate win and loose percentages. Win and loose should be equal. This will also be our sanity check. Because the win and loose probabilities are estimated by a network, they will have some error. So we will use the mean of both as estimator for the strength of average hands. Win-loose- and split percenteges are euqivalent, because they can be calculated from each other. 
Beyond the avg-handStrength, we are interested in a variance meassure of hand strengths. So we will calculate standard deviations for win, and split-pots. The output will therefore be an average value for win/loos, an std for win/loss and a std for splits.

In [8]:
#load model for generating flop strengths.
modelFlop = load_model(modelsFolder+'strengthFlopCorrected.h5')

2022-01-13 20:21:51.773298: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-13 20:21:51.773340: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-13 20:21:51.773375: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cromi-Lenovo-V15-ADA): /proc/driver/nvidia/version does not exist
2022-01-13 20:21:51.773816: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
#Function takes in a row from the data frame and calculates pwin/ploose, stdwin, stdloose

def strengthVillain(row):
    import numpy as np
    heroHand = transformHand(row)
    deckRem = set(deck) -set(heroHand)
    handsVillain= []
    for combo in combinations(deckRem, 2):
        villainHole = list(combo)
        villainHand = np.array(villainHole + heroHand[2:])
        handsVillain.append(list(villainHand.flatten()))
    handsVillain = np.array(handsVillain)
    strengths = modelFlop.predict(handsVillain)
    pwin = strengths[:,0].mean()
    ploose = strengths[:,1].mean()
    psplit = 1-(pwin+ploose)
    stdwin = (strengths[:,0].std()+strengths[:,1].std())/2
    stdsplit = strengths[:,2].std()
    return pwin, ploose, psplit, stdwin, stdsplit

In [ ]:
results = []
for i in simHandsFlopLarge.index:
    if i%100==0:
        print(i)
    result = list(strengthVillain(simHandsFlopLarge.iloc[i]))
    results.append(result)

In [23]:
pd.DataFrame(columns=['pwin', 'ploose', 'psplit', 'stdwin', 'stdsplit'], data =np.array(results))

,pwin,ploose,psplit,stdwin,stdsplit
0,0.485143,0.493260,0.021597,0.165436,0.007064
1,0.470235,0.497450,0.032316,0.184752,0.013901
2,0.493930,0.481355,0.024715,0.183468,0.009823
3,0.485079,0.490257,0.024664,0.183816,0.008695
4,0.469364,0.508005,0.022631,0.167758,0.007361
...,...,...,...,...,...
239995,0.456994,0.477016,0.065991,0.213797,0.032440
239996,0.508886,0.458892,0.032222,0.205281,0.013989
239997,0.480299,0.480298,0.039404,0.187534,0.014373
239998,0.470531,0.497846,0.031623,0.186471,0.010017


In [24]:
results = pd.DataFrame(columns=['pwin', 'ploose', 'psplit', 'stdwin', 'stdsplit'], data =np.array(results))
results.to_csv(dataFolder+'results.csv', index = None)

<h2>Final DataFrame for Training the Full Strength Evaluator for the Flop </h2>
Now let's add all values that simulated to the data frame, so we can train a model to predict those values. We will add pwin and ploose for hero, as the probabity fo split is equivalent (1-pwin-ploose). We will add the probability of split pot  for villain (the probability for pwin and ploose is equivalent), the standard deviation for the aggregate pwinVil and plooseVil and a standard deviation for split pot.

In [87]:
simHandsFlopTotal = simHandsFlopLarge
simHandsFlopTotal['psplitVil']=results['psplit']
simHandsFlopTotal['stdwinVil']=results['stdwin']
simHandsFlopTotal['stdsplit']=results['stdsplit']
simHandsFlopTotal.rename(columns={'nwin':'pwin', 'nloose':'ploose', 'stdsplit':'stdsplitVil'}, inplace=True)
simHandsFlopTotal.to_csv(dataFolder+'simHandsFlopTotal.csv', index=None)

simHandsFlopTotal.head()

,val1,suit1,val2,suit2,f1val,f1suit,f2val,f2suit,f3val,f3suit,pwin,ploose,nsplit,psplitVil,stdwinVil,stdsplitVil
0,13,3,2,1,3,3,5,1,8,0,0.339,0.632,0.029,0.021597,0.165436,0.007064
1,12,2,8,3,6,3,12,0,7,2,0.809,0.178,0.013,0.032316,0.184752,0.013901
2,2,0,6,1,5,0,12,3,4,3,0.279,0.692,0.029,0.024715,0.183468,0.009823
3,5,2,3,3,2,3,14,2,4,2,0.914,0.046,0.040,0.024664,0.183816,0.008695
4,9,2,12,0,9,1,3,0,6,2,0.754,0.237,0.009,0.022631,0.167758,0.007361


In [12]:
simHandsFlopTotal.to_csv(dataFolder+'simHandsFlopTotal.csv', index=None)
simHandsFlopTotal.head()

,val1,suit1,val2,suit2,f1val,f1suit,f2val,f2suit,f3val,f3suit,pwin,ploose,psplitVil,stdwinVil,stdsplitVil
0,13,3,2,1,3,3,5,1,8,0,0.339,0.632,0.021597,0.165436,0.007064
1,12,2,8,3,6,3,12,0,7,2,0.809,0.178,0.032316,0.184752,0.013901
2,2,0,6,1,5,0,12,3,4,3,0.279,0.692,0.024715,0.183468,0.009823
3,5,2,3,3,2,3,14,2,4,2,0.914,0.046,0.024664,0.183816,0.008695
4,9,2,12,0,9,1,3,0,6,2,0.754,0.237,0.022631,0.167758,0.007361


In [4]:
simHandsFlopTotal= pd.read_csv(dataFolder+'simHandsFlopTotal.csv')
simHandsFlopTotal.head()

,val1,suit1,val2,suit2,f1val,f1suit,f2val,f2suit,f3val,f3suit,pwin,ploose,psplitVil,stdwinVil,stdsplitVil
0,13,3,2,1,3,3,5,1,8,0,0.339,0.632,0.021597,0.165436,0.007064
1,12,2,8,3,6,3,12,0,7,2,0.809,0.178,0.032316,0.184752,0.013901
2,2,0,6,1,5,0,12,3,4,3,0.279,0.692,0.024715,0.183468,0.009823
3,5,2,3,3,2,3,14,2,4,2,0.914,0.046,0.024664,0.183816,0.008695
4,9,2,12,0,9,1,3,0,6,2,0.754,0.237,0.022631,0.167758,0.007361


In [32]:
input_dims = 10

fc1_dims = 256
fc2_dims = 128
fc3_dims = 64
fc4_dims = 32
fc5_dims = 16
fc6_dims = 8

lr = 0.001
strengthFlopTotal = keras.Sequential()
strengthFlopTotal.add(keras.layers.Dense(fc1_dims, input_dim=input_dims, activation = 'relu'))
strengthFlopTotal.add(keras.layers.Dense(fc2_dims, activation = 'relu'))
strengthFlopTotal.add(keras.layers.Dense(fc3_dims, activation = 'relu'))
strengthFlopTotal.add(keras.layers.Dense(fc4_dims, activation = 'relu'))
strengthFlopTotal.add(keras.layers.Dense(fc5_dims, activation = 'relu'))
strengthFlopTotal.add(keras.layers.Dense(fc6_dims, activation = 'relu'))
strengthFlopTotal.add(keras.layers.Dense(5,activation='sigmoid'))
strengthFlopTotal.compile(optimizer=Adam(learning_rate=lr), loss='mean_squared_error') #We use the absolute errors to reduce the effect of scale, 
#because the predicted values are on quite different scales. However as indicators they should be equally important. Because each deviation from the true value is divided by
#the true value, the error is properly scaled.

In [5]:
from sklearn.model_selection import train_test_split
seed = 1
train, test = train_test_split(simHandsFlopTotal, test_size=0.05, random_state = seed)

In [6]:
from sklearn.preprocessing import MinMaxScaler
X_train = np.array(train.iloc[:,0:-5])
X_test = np.array(test.iloc[:,0:-5])

scaler = MinMaxScaler()
y = np.array(simHandsFlopTotal.iloc[:,-5:])
scaler.fit(y[:,2:])

y_train = np.array(train.iloc[:,-5:])
y_test = np.array(test.iloc[:,-5:])


y_train[:,2:] = scaler.transform(y_train[:,2:])
y_test[:,2:] = scaler.transform(y_test[:,2:])


In [35]:
strengthFlopTotal.fit(X_train,y_train, batch_size=8, epochs=10)

Epoch 1/10
28500/28500 [==============================] - 83s 3ms/step - loss: 0.0135
Epoch 2/10
28500/28500 [==============================] - 76s 3ms/step - loss: 0.0075
Epoch 3/10
28500/28500 [==============================] - 76s 3ms/step - loss: 0.0051
Epoch 4/10
28500/28500 [==============================] - 76s 3ms/step - loss: 0.0042
Epoch 5/10
28500/28500 [==============================] - 72s 3ms/step - loss: 0.0036
Epoch 6/10
28500/28500 [==============================] - 74s 3ms/step - loss: 0.0032
Epoch 7/10
28500/28500 [==============================] - 74s 3ms/step - loss: 0.0030
Epoch 8/10
28500/28500 [==============================] - 79s 3ms/step - loss: 0.0028
Epoch 9/10
28500/28500 [==============================] - 81s 3ms/step - loss: 0.0025
Epoch 10/10
28500/28500 [==============================] - 81s 3ms/step - loss: 0.0024


In [9]:
strengthFlopTotal = load_model(modelsFolder+'strengthFlop3.h5')
lr = 0.0000001
K.set_value(strengthFlopTotal.optimizer.learning_rate, lr)
strengthFlopTotal.fit(X_train,y_train, batch_size=8, epochs=20)

In [22]:
strengthFlopTotal.predict(X_test[:25])

array([[0.46765143, 0.4945189 , 0.12524655, 0.8006493 , 0.13909328],
       [0.4719491 , 0.49321872, 0.09356117, 0.15850863, 0.08841214],
       [0.5085791 , 0.46261612, 0.0907912 , 0.64106786, 0.10879248],
       [0.21485916, 0.73947364, 0.0757111 , 0.6568994 , 0.09358349],
       [0.6165437 , 0.35924232, 0.13892621, 0.4585023 , 0.15048093],
       [0.41519517, 0.5324584 , 0.20666817, 0.85706687, 0.27256003],
       [0.4175225 , 0.53879267, 0.17733195, 0.30941465, 0.17821762],
       [0.4983852 , 0.46148378, 0.21150067, 0.53317666, 0.2375401 ],
       [0.7223859 , 0.25202143, 0.12987718, 0.6554334 , 0.19415978],
       [0.7522038 , 0.22609267, 0.14251   , 0.42018616, 0.15645728],
       [0.49323544, 0.45481265, 0.24805164, 0.61019206, 0.24700302],
       [0.24783725, 0.7114421 , 0.06937665, 0.69918764, 0.08796927],
       [0.44507438, 0.5182116 , 0.17133051, 0.7478094 , 0.2351912 ],
       [0.2592069 , 0.6454494 , 0.32315063, 0.6698858 , 0.38267425],
       [0.38058308, 0.58644307, 0.

In [18]:
y_test[:25]

array([[0.532     , 0.437     , 0.1545432 , 0.79968491, 0.19345065],
       [0.441     , 0.5304    , 0.0996646 , 0.17730845, 0.1013733 ],
       [0.5242    , 0.4618    , 0.08812387, 0.67930765, 0.11025361],
       [0.227     , 0.733     , 0.06932468, 0.65352791, 0.10754643],
       [0.6408    , 0.3062    , 0.14363006, 0.43058582, 0.15024219],
       [0.4148    , 0.5266    , 0.21325468, 0.86368483, 0.26435405],
       [0.4       , 0.572     , 0.18309297, 0.35312869, 0.17433681],
       [0.459     , 0.516     , 0.231039  , 0.48386673, 0.20988072],
       [0.747     , 0.252     , 0.14206513, 0.62996358, 0.1824833 ],
       [0.72      , 0.266     , 0.15822628, 0.39423121, 0.16994382],
       [0.413     , 0.545     , 0.23823608, 0.58800688, 0.25090705],
       [0.236     , 0.731     , 0.07606115, 0.68644037, 0.11743944],
       [0.432     , 0.555     , 0.15325821, 0.78889157, 0.22329292],
       [0.248     , 0.669     , 0.31379593, 0.65506817, 0.41323716],
       [0.379     , 0.581     , 0.

In [9]:
strengthFlopTotal.predict(X_test[:25])

array([[0.469941  , 0.49257642, 0.12447572, 0.7982395 , 0.13852832],
       [0.46839938, 0.4968017 , 0.09336692, 0.16268763, 0.08886886],
       [0.5079062 , 0.4636618 , 0.09084234, 0.64244974, 0.10894766],
       [0.21448487, 0.73975635, 0.07565799, 0.65737665, 0.09363404],
       [0.6215425 , 0.3548059 , 0.1379134 , 0.4572317 , 0.14984205],
       [0.41559362, 0.53126514, 0.20720497, 0.85579026, 0.27484807],
       [0.41991514, 0.5357667 , 0.17817   , 0.3071716 , 0.1798681 ],
       [0.4972922 , 0.4611161 , 0.21595412, 0.52785265, 0.24081066],
       [0.7213243 , 0.25309664, 0.12935272, 0.65630084, 0.19319403],
       [0.7457385 , 0.2314308 , 0.14321238, 0.41996038, 0.1565718 ],
       [0.49642316, 0.45327744, 0.2494129 , 0.6110247 , 0.24698055],
       [0.2386029 , 0.71896815, 0.06964126, 0.6946989 , 0.08838269],
       [0.44792336, 0.5140221 , 0.17442006, 0.7518139 , 0.2422013 ],
       [0.26026434, 0.6446115 , 0.32464138, 0.6697929 , 0.3813444 ],
       [0.3801844 , 0.5870614 , 0.

In [81]:
X_test[10:20]

array([[11,  3, 13,  3,  7,  0,  9,  1, 14,  1],
       [ 9,  0, 10,  3,  2,  2,  6,  1, 13,  1],
       [13,  3,  7,  0,  3,  3,  5,  0, 13,  2],
       [ 9,  3,  8,  0,  3,  2, 11,  2,  3,  3],
       [11,  3, 11,  1,  8,  1,  3,  1,  9,  2],
       [ 8,  3, 12,  2,  8,  1, 11,  3,  6,  3],
       [ 6,  3,  8,  3,  9,  1, 14,  1, 14,  3],
       [12,  0,  8,  3,  2,  0,  2,  2,  9,  1],
       [ 6,  1,  7,  2,  2,  0,  9,  2, 12,  0],
       [ 3,  2, 13,  2,  3,  0,  7,  3, 11,  3]])

In [16]:
strengthFlopTotal.evaluate(X_test,y_test)

375/375 [==============================] - 1s 2ms/step - loss: 0.0015


0.001520959660410881

In [21]:
strengthFlopTotal.save(modelsFolder+'strengthFlop3.h5')